<a href="https://colab.research.google.com/github/mijin0721/Sentiment_analysis_AI/blob/main/%EA%B2%B0%EA%B3%BC%EB%8F%84%EC%B6%9C_No1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-is9a8lqw/kobert-tokenizer_9ce81d315f304c57a3bab1c4c169e0a6
  Running command git clone -q https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-is9a8lqw/kobert-tokenizer_9ce81d315f304c57a3bab1c4c169e0a6


In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-liyuk4vu
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-liyuk4vu
  Using cached transformers-4.8.1-py3-none-any.whl (2.5 MB)
  Using cached tokenizers-0.10.3-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (3.3 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.8.1rc1
    Uninstalling tokenizers-0.8.1rc1:
      Successfully uninstalled tokenizers-0.8.1rc1
  Attempting uninstall: transformers
    Found existing installation: transformers 3.0.2
    Uninstalling transformers-3.0.2:
      Successfully uninstalled transformers-3.0.2


In [ ]:
# torch
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#GPU 사용
device = torch.device("cuda:0")

In [ ]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
# KoBERT에 입력될 데이터셋 정리
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels)) 

In [ ]:
# 모델 정의
class BERTClassifier(nn.Module): ## 클래스를 상속
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)


In [ ]:
# Setting parameters
max_len = 64
batch_size = 32
warmup_ratio = 0.1
num_epochs = 20
max_grad_norm = 1
log_interval = 100
learning_rate =  5e-5

In [ ]:
## 학습 모델 로드
PATH = '/content/drive/MyDrive/Colab Notebooks'
model = torch.load(PATH + 'KoBERT_담화.pt')  # 전체 모델을 통째로 불러옴, 클래스 선언 필수
model.load_state_dict(torch.load(PATH + 'model_state_dict.pt'))  # state_dict를 불러 온 후, 모델에 저장


<All keys matched successfully>

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def new_softmax(a) : 
    c = np.max(a) # 최댓값
    exp_a = np.exp(a-c) # 각각의 원소에 최댓값을 뺀 값에 exp를 취한다. (이를 통해 overflow 방지)
    sum_exp_a = np.sum(exp_a)
    y = (exp_a / sum_exp_a) * 100
    return np.round(y, 3)


using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)

        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()
            min_v = min(logits)
            total = 0
            probability = []
            logits = np.round(new_softmax(logits), 3).tolist()
            for logit in logits:
                print(logit)
                probability.append(np.round(logit, 3))

            if np.argmax(logits) == 0:  emotion = "0"
            elif np.argmax(logits) == 1: emotion = "1"

            probability.append(emotion)
            print(probability)
    return probability

In [ ]:
import pandas as pd
data = pd.read_csv('test_data.csv', encoding='utf-8')
data

FileNotFoundError: ignored

In [ ]:
for i in range(0, data.shape[0]):
    row = data.iloc[i]
    idx = data.index[i]
    print(f'행 위치 : {i} 번째')
    print('{}', row['굳 ㅋ'])

행 위치 : 0 번째
{} GDNTOPCLASSINTHECLUB
행 위치 : 1 번째
{} 뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아
행 위치 : 2 번째
{} 지루하지는 않은데 완전 막장임... 돈주고 보기에는....
행 위치 : 3 번째
{} 3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??


In [ ]:
df = [[0.0,0.0,0.0]]
for i in range(0, data.shape[0]):
    row = data.iloc[i]
    idx = data.index[i]
    sentence = row['review']
    result = predict(sentence)
    df.append(result)

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


35.744998931884766
64.25499725341797
[35.745, 64.255, '1']
49.04399871826172
50.95600128173828
[49.044, 50.956, '1']
92.45600128173828
7.544000148773193
[92.456, 7.544, '0']
77.16699981689453
22.83300018310547
[77.167, 22.833, '0']
90.73100280761719
9.269000053405762
[90.731, 9.269, '0']
57.702999114990234
42.297000885009766
[57.703, 42.297, '0']
66.4489974975586
33.55099868774414
[66.449, 33.551, '0']
62.29600143432617
37.70399856567383
[62.296, 37.704, '0']
84.03800201416016
15.961999893188477
[84.038, 15.962, '0']
81.36900329589844
18.631000518798828
[81.369, 18.631, '0']
43.297000885009766
56.702999114990234
[43.297, 56.703, '1']
45.62799835205078
54.37200164794922
[45.628, 54.372, '1']
72.66999816894531
27.329999923706055
[72.67, 27.33, '0']
65.64600372314453
34.354000091552734
[65.646, 34.354, '0']
88.65799713134766
11.342000007629395
[88.658, 11.342, '0']
68.75800323486328
31.242000579833984
[68.758, 31.242, '0']
18.524999618530273
81.4749984741211
[18.525, 81.475, '1']
38.24200

In [ ]:
df

[[0.0, 0.0, 0.0],
 [35.745, 64.255, '1'],
 [49.044, 50.956, '1'],
 [92.456, 7.544, '0'],
 [77.167, 22.833, '0'],
 [90.731, 9.269, '0'],
 [57.703, 42.297, '0'],
 [66.449, 33.551, '0'],
 [62.296, 37.704, '0'],
 [84.038, 15.962, '0'],
 [81.369, 18.631, '0'],
 [43.297, 56.703, '1'],
 [45.628, 54.372, '1'],
 [72.67, 27.33, '0'],
 [65.646, 34.354, '0'],
 [88.658, 11.342, '0'],
 [68.758, 31.242, '0'],
 [18.525, 81.475, '1'],
 [38.242, 61.758, '1'],
 [59.932, 40.068, '0'],
 [85.934, 14.066, '0'],
 [83.224, 16.776, '0'],
 [35.431, 64.569, '1'],
 [13.993, 86.007, '1'],
 [67.303, 32.697, '0'],
 [67.872, 32.128, '0'],
 [68.991, 31.009, '0'],
 [80.411, 19.589, '0'],
 [84.078, 15.922, '0'],
 [22.659, 77.341, '1'],
 [55.171, 44.829, '0'],
 [51.365, 48.635, '0'],
 [94.284, 5.716, '0'],
 [50.568, 49.432, '0'],
 [83.488, 16.512, '0'],
 [84.586, 15.414, '0'],
 [64.993, 35.007, '0'],
 [85.405, 14.595, '0'],
 [47.774, 52.226, '1'],
 [86.612, 13.388, '0'],
 [29.661, 70.339, '1'],
 [85.339, 14.661, '0'],
 [9

In [ ]:
df_last=pd.DataFrame(df,columns=['col1','col2','result'])
df_last.drop(0, axis=0)
df_last = df_last['result']

In [ ]:
df_last.to_excel('/content/drive/MyDrive/test.xlsx') #csv파일로 생성